In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.impute import SimpleImputer


In [42]:
#read datasets
bankusersdf = pd.read_csv('churn.csv')

In [43]:
#explore first 5 rows
bankusersdf.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [44]:
#check for type by uitlizing .info()
bankusersdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [45]:
#check for duplicates in custoemr id

duplicate_count = bankusersdf.duplicated(subset=['CustomerId']).sum()
print(duplicate_count)



0


In [46]:
missing_values = bankusersdf.isnull().sum()
print(missing_values)


RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64


In [47]:
# Display rows with missing values
missing_rows = bankusersdf[bankusersdf.isnull().any(axis=1)]
missing_rows


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
30,31,15589475,Azikiwe,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,49,15766205,Yin,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,52,15768193,Trevisani,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,54,15702298,Parkhill,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,61,15651280,Hunter,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,9945,15703923,Cameron,744,Germany,Male,41,NaN,190409.34,2,1,1,138361.48,0
9956,9957,15707861,Nucci,520,France,Female,46,NaN,85216.61,1,1,0,117369.52,1
9964,9965,15642785,Douglas,479,France,Male,34,NaN,117593.48,2,0,0,113308.29,0
9985,9986,15586914,Nepean,659,France,Male,36,NaN,123841.49,2,1,0,96833.00,0


# Tenure is period of maturation for a customer’s fixed deposit (years). So this means these account holders do not have deposits with the bank. I won't edit the data with these users for now.

In [48]:

#Find some statical values for the dataset
bankusersdf.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


# Examine the balance of classes. Train the model without taking into account the imbalance. Briefly describe your findings.

In [49]:


# Select relevant features and target variable
X = bankusersdf.drop(columns=['RowNumber', 'CustomerId', 'Surname', 'Exited'])
y = bankusersdf['Exited']



In [50]:
# Encode categorical variables
label_encoders = {}
for col in ['Geography', 'Gender']:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le



In [51]:

# Handle missing values if any
X = X.fillna(X.median())

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)



In [52]:


# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Train logistic regression model
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)



LogisticRegression(random_state=42)

In [53]:
# Make predictions
y_pred = model.predict(X_test)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)

Accuracy: 0.8045
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.97      0.89      1593
           1       0.58      0.14      0.23       407

    accuracy                           0.80      2000
   macro avg       0.70      0.56      0.56      2000
weighted avg       0.77      0.80      0.75      2000



# The logistic regression model achieved an accuracy of 80.45%, but the performance on the minority class (Exited = 1) is poor:

    Precision for Exited (1): 58%
    Recall for Exited (1): 14%
    F1-score for Exited (1): 23%

# This indicates that the model is biased towards the majority class, as it fails to correctly identify most of the exited customers. The recall for the minority class is very low, meaning many actual exits are misclassified as non-exits.


# Improve the quality of the model. Make sure you use at least two approaches to fixing class imbalance. Use the training set to pick the best parameters. Train different models on training and validation sets. Find the best one. Briefly describe your findings.

In [54]:
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)



In [55]:
# Address class imbalance using oversampling
X_train_resampled, y_train_resampled = resample(X_train[y_train == 1], y_train[y_train == 1],
                                                replace=True, n_samples=len(y_train[y_train == 0]), random_state=42)
X_train_balanced = np.vstack((X_train[y_train == 0], X_train_resampled))
y_train_balanced = np.hstack((y_train[y_train == 0], y_train_resampled))

# Train logistic regression with hyperparameter tuning
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
logistic_model = GridSearchCV(LogisticRegression(random_state=42), param_grid, cv=5, scoring='f1')
logistic_model.fit(X_train_balanced, y_train_balanced)

# Train random forest classifier with hyperparameter tuning
param_grid_rf = {'n_estimators': [50, 100, 200], 'max_depth': [50, 100, None]}
rf_model = GridSearchCV(RandomForestClassifier(random_state=12345), param_grid_rf, cv=5, scoring='f1')
rf_model.fit(X_train_balanced, y_train_balanced)



GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=12345),
             param_grid={'max_depth': [50, 100, None],
                         'n_estimators': [50, 100, 200]},
             scoring='f1')

In [56]:
# Evaluate models
y_pred_logistic = logistic_model.best_estimator_.predict(X_test)
y_pred_rf = rf_model.best_estimator_.predict(X_test)

accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
accuracy_rf = accuracy_score(y_test, y_pred_rf)

report_logistic = classification_report(y_test, y_pred_logistic)
report_rf = classification_report(y_test, y_pred_rf)

# Select the best model
best_model = logistic_model.best_estimator_ if accuracy_logistic > accuracy_rf else rf_model.best_estimator_

print(f"Best Model: {'Logistic Regression' if best_model == logistic_model.best_estimator_ else 'Random Forest'}")
print(f"Logistic Regression Accuracy: {accuracy_logistic:.4f}")
print("Logistic Regression Classification Report:")
print(report_logistic)
print(f"Random Forest Accuracy: {accuracy_rf:.4f}")
print("Random Forest Classification Report:")
print(report_rf)

Best Model: Random Forest
Logistic Regression Accuracy: 0.7070
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.70      0.79      1593
           1       0.38      0.71      0.50       407

    accuracy                           0.71      2000
   macro avg       0.64      0.71      0.65      2000
weighted avg       0.80      0.71      0.73      2000

Random Forest Accuracy: 0.8480
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.93      0.91      1593
           1       0.66      0.51      0.58       407

    accuracy                           0.85      2000
   macro avg       0.77      0.72      0.74      2000
weighted avg       0.84      0.85      0.84      2000



In [57]:
# Initialize the imputer
imputer = SimpleImputer(strategy='mean')

# Impute missing values in X
X_imputed = imputer.fit_transform(X)

# Standardize the features after imputation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)


In [58]:
# Drop rows with any missing values
df_clean = df.dropna()

# Redefine features and target after cleaning
X = df_clean.drop(columns=['Exited'])
y = df_clean['Exited']

# Proceed with encoding, scaling, splitting, etc.


# Perform the final testing

In [64]:
# Train random forest classifier with hyperparameter tuning
param_grid_rf = {'n_estimators': [50, 100, 200], 'max_depth': [5, 10, None]}
rf_model = GridSearchCV(RandomForestClassifier(random_state=12345), param_grid_rf, cv=5, scoring='f1')
rf_model.fit(X_train_balanced, y_train_balanced)

# Final testing on the best random forest model
y_pred_rf = rf_model.best_estimator_.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
report_rf = classification_report(y_test, y_pred_rf)

print("Final Testing on Random Forest Model:")
print(f"Accuracy: {accuracy_rf:.4f}")
print("Classification Report:")
print(report_rf)

# Compute AUC-ROC
auc_roc = roc_auc_score(y_test, y_pred_rf)
print(f"AUC-ROC Score: {auc_roc:.4f}")

# Compute F1-Score
f1 = f1_score(y_test, y_pred_rf)
print(f"F1 Score: {f1:.4f}")


Final Testing on Random Forest Model:
Accuracy: 0.8480
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.93      0.91      1593
           1       0.66      0.51      0.58       407

    accuracy                           0.85      2000
   macro avg       0.77      0.72      0.74      2000
weighted avg       0.84      0.85      0.84      2000

AUC-ROC Score: 0.7235
F1 Score: 0.5789


# Having a hard time getting the f1 score over .59

In [59]:
# Address class imbalance using oversampling
X_test_resampled, y_test_resampled = resample(X_test[y_test == 1], y_test[y_test == 1],
                                                replace=True, n_samples=len(y_test[y_test == 0]), random_state=42)
X_test_balanced = np.vstack((X_test[y_test == 0], X_test_resampled))
y_test_balanced = np.hstack((y_test[y_test == 0], y_test_resampled))



In [60]:
# Train random forest classifier with hyperparameter tuning
param_grid_rf = {'n_estimators': [50, 100, 200], 'max_depth': [5, 10, None]}
rf_model = GridSearchCV(RandomForestClassifier(random_state=12345), param_grid_rf, cv=5, scoring='f1')
rf_model.fit(X_train_balanced, y_train_balanced)



GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=12345),
             param_grid={'max_depth': [5, 10, None],
                         'n_estimators': [50, 100, 200]},
             scoring='f1')

In [61]:


# Final testing on the best random forest model
y_pred_rf = rf_model.best_estimator_.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
report_rf = classification_report(y_test, y_pred_rf)



In [62]:
print("Final Testing on Random Forest Model:")
print(f"Accuracy: {accuracy_rf:.4f}")
print("Classification Report:")
print(report_rf)


Final Testing on Random Forest Model:
Accuracy: 0.8480
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.93      0.91      1593
           1       0.66      0.51      0.58       407

    accuracy                           0.85      2000
   macro avg       0.77      0.72      0.74      2000
weighted avg       0.84      0.85      0.84      2000



In [63]:
# Compute AUC-ROC
auc_roc = roc_auc_score(y_test, y_pred_rf)
print(f"AUC-ROC Score: {auc_roc:.4f}")

# Compute F1-Score
f1 = f1_score(y_test, y_pred_rf)
print(f"F1 Score: {f1:.4f}")

AUC-ROC Score: 0.7235
F1 Score: 0.5789


In [65]:


smote_tomek = SMOTETomek(random_state=42)
X_train_balanced, y_train_balanced = smote_tomek.fit_resample(X_train, y_train)


In [66]:
# Train random forest classifier with hyperparameter tuning
param_grid_rf = {'n_estimators': [50, 100, 200], 'max_depth': [5, 10, None]}
rf_model = GridSearchCV(RandomForestClassifier(random_state=12345), param_grid_rf, cv=5, scoring='f1')
rf_model.fit(X_train_balanced, y_train_balanced)

# Final testing on the best random forest model
y_pred_rf = rf_model.best_estimator_.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
report_rf = classification_report(y_test, y_pred_rf)

print("Final Testing on Random Forest Model:")
print(f"Accuracy: {accuracy_rf:.4f}")
print("Classification Report:")
print(report_rf)

# Compute AUC-ROC
auc_roc = roc_auc_score(y_test, y_pred_rf)
print(f"AUC-ROC Score: {auc_roc:.4f}")

# Compute F1-Score
f1 = f1_score(y_test, y_pred_rf)
print(f"F1 Score: {f1:.4f}")

Final Testing on Random Forest Model:
Accuracy: 0.8380
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.91      0.90      1593
           1       0.61      0.56      0.58       407

    accuracy                           0.84      2000
   macro avg       0.75      0.73      0.74      2000
weighted avg       0.83      0.84      0.84      2000

AUC-ROC Score: 0.7346
F1 Score: 0.5846


In [67]:


# Get prediction probabilities for the positive class
y_prob_rf = rf_model.best_estimator_.predict_proba(X_test)[:, 1]

# Find the optimal threshold that maximizes F1
best_threshold = 0.5
best_f1 = 0
for thresh in np.arange(0.3, 0.7, 0.01):
    y_pred_thresh = (y_prob_rf >= thresh).astype(int)
    score = f1_score(y_test, y_pred_thresh)
    if score > best_f1:
        best_f1 = score
        best_threshold = thresh

print(f"Optimal threshold: {best_threshold:.2f} with F1: {best_f1:.4f}")

# Use the optimal threshold for final predictions
y_pred_rf_adjusted = (y_prob_rf >= best_threshold).astype(int)
final_f1 = f1_score(y_test, y_pred_rf_adjusted)
print(f"Final F1 Score after threshold tuning: {final_f1:.4f}")


Optimal threshold: 0.44 with F1: 0.5955
Final F1 Score after threshold tuning: 0.5955
